In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import itertools as itt
from ipywidgets import widgets
import ConfigReader as cr

/home/mendozah/workspace/envHDMp/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
idx = pd.IndexSlice

#Styles
sns.set_style('darkgrid')
sns.set_color_codes()
plt.rcParams['figure.figsize'] = (12.0, 9.0)

plt.rc('text', usetex=False)
plt.rc('font', size=20.0, family='serif')

In [8]:
# key -> dataset code, name -> dataset name
dataset_dict = {'1128_bac': 'OVA_Breast', '554_bac':'MNIST', '46_bac':'Splice'}

In [9]:
[dataset_dict.get(k) for k in dataset_dict]

['MNIST', 'OVA_Breast', 'Splice']

In [ ]:
# From my opinion very sloppy way to do it
clear_name = lambda X: X.split(':')[-1] if X.split(':')[0] == 'classifier' else X.split(':')[0]

In [10]:
base_data_dir = '/mhome/mendozah/constrained_DeepNet_Configs/'
solvers = ['adam', 'adadelta', 'adagrad', 'nesterov', 'sgd', 'momentum']
solvers_dirs = list(map(lambda X: X + '_results', solvers))
policies = ['fixed', 'inv', 'exp', 'step']
interesting_params = [ 'response', 'solver', 'learning_rate', 'beta1', 'beta2', 'momentum',
                      'lambda2', 'lr_policy','gamma', 'power', 'epoch_step']

## Parameter importance plotting

In [181]:
import pyfanova.fanova

In [182]:
reload(pyfanova.fanova)

<module 'pyfanova.fanova' from '/home/mendozah/workspace/fanova/pyfanova/fanova.pyc'>

In [183]:
state_run_dir = "/mhome/mendozah/constrained_DeepNet_Configs/adam_results/MNIST/inv/merged_runs_inv/"

In [184]:
fanova_sets = dict()

for policy in policies:
    solver_run_dir = os.path.join(base_data_dir, solvers_dirs[0],
                              dataset_dict.get('554_bac'),
                              policy, 'merged_runs_' + policy)
    fanova_sets[policy] = pyfanova.fanova.Fanova(solver_run_dir)

In [186]:
max_marginals = 5
cols_imp_df = ['marginal', 'parameter']

In [49]:
assert len(fanova_sets) == len(policies), "Not all policies are available"

In [187]:
policy_marginals = []

for k, v in fanova_sets.items():
    temp_df = (pd.DataFrame(v.print_all_marginals(max_num=max_marginals, pairwise=True), columns=cols_imp_df)
               .assign(policy=k))
    policy_marginals.append(temp_df)
    print("======================= DONE =======================")

In [189]:
plot_df = pd.concat(policy_marginals).reset_index(drop=True)
plot_df.loc[:, ['parameter']] = plot_df.parameter.str.split(' x ')

In [ ]:
for name, group in plot_df.groupby('policy'):
    print name
    for g in group.parameter:
        print("ECA %s" % map(lambda X: X.split(':')[-1] if X.split(':')[0] == 'classifier' else X.split(':')[0], g))

### Plot now the marginals. Columns is the order and row is the policy

In [174]:
import pyfanova.visualizer

In [240]:
reload(pyfanova.visualizer)

<module 'pyfanova.visualizer' from '/home/mendozah/workspace/fanova/pyfanova/visualizer.py'>

In [241]:
viz_sets = {k: pyfanova.visualizer.Visualizer(v) for k,v in fanova_sets.items()}

In [ ]:
fig_hyper, axh = plt.subplots(nrows=len(viz_sets), ncols=max_marginals, figsize=(32, 20.), sharey=True)
for row, (name, group) in zip(axh, plot_df.groupby('policy')):
    key_viz = viz_sets[name]
    categorical_params = fanova_sets[name].get_config_space().get_categorical_parameters()
    for ax, p in zip(row, group.parameter):
        if len(p) == 1:
            if p[0] not in categorical_params:
                key_viz.plot_marginal(p[0], ax=ax)
            else:
                key_viz.plot_categorical_marginal(p[0],ax=ax)    
            ax.set_xlabel(clear_name(p[0]))
        else:
            if (p[0] in categorical_params) and (p[1] not in categorical_params):
                key_viz.plot_contour_categorical_pairwise(p[0], p[1], ax=ax)
                ax.set_xlabel(clear_name(p[1]))
        ax.set_title('Policy: ' + name)
plt.tight_layout()
fig_hyper.savefig(os.path.join('images', str(max_marginals) + '_Most_Important_Params_on_MNIST.png'))

In [ ]:
s = len(importance_params)
fig_effect, axp = plt.subplots(nrows=int(np.ceil(s/3.)), ncols=3, figsize=(15., 12.), sharey=True)
for ax, name in zip(axp.flat, importance_params):
    viz_MNIST.plot_marginal('classifier:AdamInvConstFeedNet:' + name, ax=ax)
    ax.set_xlabel(name)
fig_effect.suptitle('Hyperparameter Importance')
#fig_effect.savefig('images_'+ dataset_name +'joint_distro_lr_response.png')
plt.tight_layout()

In [ ]:
ax_params = viz_MNIST.plot_pairwise_marginal('classifier:AdamInvConstFeedNet:beta1',
                                             'classifier:AdamInvConstFeedNet:lambda2', resolution=50)

In [ ]:
s = len(importance_params)
fig_interaction, axi = plt.subplots(nrows=int(np.ceil(s/3.)), ncols=3, figsize=(22., 15.), sharey=True)
for ax, name in zip(axi.flat, importance_params):
    viz_MNIST.plot_contour_pairwise('classifier:AdamInvConstFeedNet:' + name,
                                    'classifier:AdamInvConstFeedNet:lambda2',
                                    bounds_1=(0.0, 1.0), bounds_2=(0.0, 1.0),
                                    log_scale_1=False, log_scale_2=True,
                                    resolution=10, ax=ax)
    ax.set_ylabel('lambda2')
    ax.set_xlabel(name)
fig_interaction.suptitle('Hyperparameter Interaction')
#fig_kde.savefig('images_'+ dataset_name +'joint_distro_lr_response.png')
plt.tight_layout()